## SQL at Scale with Spark SQL

Welcome to the SQL mini project. For this project, you will use the Databricks Platform and work through a series of exercises using Spark SQL. The dataset size may not be too big but the intent here is to familiarize yourself with the Spark SQL interface which scales easily to huge datasets, without you having to worry about changing your SQL queries. 

The data you need is present in the mini-project folder in the form of three CSV files. This data will be imported in Databricks to create the following tables under the __`country_club`__ database.

<br>
1. The __`bookings`__ table,
2. The __`facilities`__ table, and
3. The __`members`__ table.

You will be uploading these datasets shortly into the Databricks platform to understand how to create a database within minutes! Once the database and the tables are populated, you will be focusing on the mini-project questions.

In the mini project, you'll be asked a series of questions. You can solve them using the databricks platform, but for the final deliverable,
please download this notebook as an IPython notebook (__`File -> Export -> IPython Notebook`__) and upload it to your GitHub.

### Creating the Database

We will first create our database in which we will be creating our three tables of interest

In [0]:
%sql 
drop database if exists country_club cascade;
create database country_club;
show databases;

databaseName
country_club
default


### Creating the Tables

In this section, we will be creating the three tables of interest and populate them with the data from the CSV files already available to you. 
To get started, first upload the three CSV files to the DBFS as depicted in the following figure

![](https://i.imgur.com/QcCruBr.png)


Once you have done this, please remember to execute the following code to build the dataframes which will be saved as tables in our database

In [0]:
# File location and type
file_location_bookings = "/FileStore/tables/Bookings.csv"
file_location_facilities = "/FileStore/tables/Facilities.csv"
file_location_members = "/FileStore/tables/Members.csv"

file_type = "csv"

# CSV options
infer_schema = "true"
first_row_is_header = "true"
delimiter = ","

# The applied options are for CSV files. For other file types, these will be ignored.
bookings_df = (spark.read.format(file_type) 
                    .option("inferSchema", infer_schema) 
                    .option("header", first_row_is_header) 
                    .option("sep", delimiter) 
                    .load(file_location_bookings))

facilities_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_facilities))

members_df = (spark.read.format(file_type) 
                      .option("inferSchema", infer_schema) 
                      .option("header", first_row_is_header) 
                      .option("sep", delimiter) 
                      .load(file_location_members))

### Viewing the dataframe schemas

We can take a look at the schemas of our potential tables to be written to our database soon

In [0]:
print('Bookings Schema')
bookings_df.printSchema()
print('Facilities Schema')
facilities_df.printSchema()
print('Members Schema')
members_df.printSchema()

Bookings Schema
root
 |-- bookid: integer (nullable = true)
 |-- facid: integer (nullable = true)
 |-- memid: integer (nullable = true)
 |-- starttime: string (nullable = true)
 |-- slots: integer (nullable = true)

Facilities Schema
root
 |-- facid: integer (nullable = true)
 |-- name: string (nullable = true)
 |-- membercost: double (nullable = true)
 |-- guestcost: double (nullable = true)
 |-- initialoutlay: integer (nullable = true)
 |-- monthlymaintenance: integer (nullable = true)

Members Schema
root
 |-- memid: integer (nullable = true)
 |-- surname: string (nullable = true)
 |-- firstname: string (nullable = true)
 |-- address: string (nullable = true)
 |-- zipcode: integer (nullable = true)
 |-- telephone: string (nullable = true)
 |-- recommendedby: integer (nullable = true)
 |-- joindate: string (nullable = true)



### Create permanent tables
We will be creating three permanent tables here in our __`country_club`__ database as we discussed previously with the following code

In [0]:
permanent_table_name_bookings = "country_club.Bookings1"
bookings_df.write.format("parquet").saveAsTable(permanent_table_name_bookings)

permanent_table_name_facilities = "country_club.Facilities1"
facilities_df.write.format("parquet").saveAsTable(permanent_table_name_facilities)

permanent_table_name_members = "country_club.Members1"
members_df.write.format("parquet").saveAsTable(permanent_table_name_members)

### Refresh tables and check them

In [0]:
%sql
use country_club;
REFRESH table bookings1;
REFRESH table facilities1;
REFRESH table members1;
show tables;

database,tableName,isTemporary
country_club,bookings1,false
country_club,facilities1,false
country_club,members1,false


### Test a sample SQL query

__Note:__ You can use __`%sql`__ at the beginning of a cell and write SQL queries directly as seen in the following cell. Neat isn't it!

In [0]:
%sql
select * from facilities1

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
2,Badminton Court,0.0,15.5,4000,50
3,Table Tennis,0.0,5.0,320,10
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80
7,Snooker Table,0.0,5.0,450,15
8,Pool Table,0.0,5.0,400,15


#### Q1: Some of the facilities charge a fee to members, but some do not. Please list the names of the facilities that do.

In [0]:
%sql
select * from facilities where (membercost!=0)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
0,Tennis Court 1,5.0,25.0,10000,200
1,Tennis Court 2,5.0,25.0,8000,200
4,Massage Room 1,9.9,80.0,4000,3000
5,Massage Room 2,9.9,80.0,4000,3000
6,Squash Court,3.5,17.5,5000,80


####  Q2: How many facilities do not charge a fee to members?

In [0]:
%sql
select count(name) as number_not_charging_members from facilities where (membercost==0)

number_not_charging_members
4


#### Q3: How can you produce a list of facilities that charge a fee to members, where the fee is less than 20% of the facility's monthly maintenance cost? 
#### Return the facid, facility name, member cost, and monthly maintenance of the facilities in question.

In [0]:
%sql
select facid, name, membercost, monthlymaintenance from facilities where (membercost!=0) and (membercost<0.2*monthlymaintenance)

facid,name,membercost,monthlymaintenance
0,Tennis Court 1,5.0,200
1,Tennis Court 2,5.0,200
4,Massage Room 1,9.9,3000
5,Massage Room 2,9.9,3000
6,Squash Court,3.5,80


#### Q4: How can you retrieve the details of facilities with ID 1 and 5? Write the query without using the OR operator.

In [0]:
%sql
select * from facilities where facid IN (1, 5)

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance
1,Tennis Court 2,5.0,25.0,8000,200
5,Massage Room 2,9.9,80.0,4000,3000


#### Q5: How can you produce a list of facilities, with each labelled as 'cheap' or 'expensive', depending on if their monthly maintenance cost is more than $100? 
#### Return the name and monthly maintenance of the facilities in question.

In [0]:
%sql
select *, (case when monthlymaintenance>100 then 'expensive' else 'cheap' end) as costliness from facilities

facid,name,membercost,guestcost,initialoutlay,monthlymaintenance,costliness
0,Tennis Court 1,5.0,25.0,10000,200,expensive
1,Tennis Court 2,5.0,25.0,8000,200,expensive
2,Badminton Court,0.0,15.5,4000,50,cheap
3,Table Tennis,0.0,5.0,320,10,cheap
4,Massage Room 1,9.9,80.0,4000,3000,expensive
5,Massage Room 2,9.9,80.0,4000,3000,expensive
6,Squash Court,3.5,17.5,5000,80,cheap
7,Snooker Table,0.0,5.0,450,15,cheap
8,Pool Table,0.0,5.0,400,15,cheap


#### Q6: You'd like to get the first and last name of the last member(s) who signed up. Do not use the LIMIT clause for your solution.

In [0]:
%sql
select firstname, surname from members1 WHERE joindate = (SELECT max(joindate) FROM members1)

firstname,surname
Darren,Smith


####  Q7: How can you produce a list of all members who have used a tennis court?
- Include in your output the name of the court, and the name of the member formatted as a single column. 
- Ensure no duplicate data
- Also order by the member name.

In [0]:
%sql
select facilities1.name, (members1.firstname, members1.surname) as mem_name from bookings1 
join facilities1 on bookings1.facid=facilities1.facid
join members1 on bookings1.memid=members1.memid
where facilities1.name like 'Tennis%'
group by mem_name, facilities1.name
order by mem_name.surname

name,mem_name
Tennis Court 1,"List(Florence, Bader)"
Tennis Court 2,"List(Florence, Bader)"
Tennis Court 1,"List(Timothy, Baker)"
Tennis Court 2,"List(Timothy, Baker)"
Tennis Court 1,"List(Anne, Baker)"
Tennis Court 2,"List(Anne, Baker)"
Tennis Court 2,"List(Tim, Boothe)"
Tennis Court 1,"List(Tim, Boothe)"
Tennis Court 1,"List(Gerald, Butters)"
Tennis Court 2,"List(Gerald, Butters)"


#### Q8: How can you produce a list of bookings on the day of 2012-09-14 which will cost the member (or guest) more than $30? 

- Remember that guests have different costs to members (the listed costs are per half-hour 'slot')
- The guest user's ID is always 0. 

#### Include in your output the name of the facility, the name of the member formatted as a single column, and the cost.

- Order by descending cost, and do not use any subqueries.

In [0]:
%sql
select facilities1.name, (members1.firstname, members1.surname) as mem_name, (case when members1.memid=0 then guestcost else membercost end)*bookings1.slots as day_cost from bookings1 
join facilities1 on bookings1.facid=facilities1.facid
join members1 on bookings1.memid=members1.memid
where bookings1.starttime > "2012-09-14" and bookings1.starttime < "2012-09-15" and (case when members1.memid=0 then guestcost else membercost end)*bookings1.slots > 30
order by (case when members1.memid=0 then guestcost else membercost end)*bookings1.slots desc

name,mem_name,day_cost
Massage Room 2,"List(GUEST, GUEST)",320.0
Massage Room 1,"List(GUEST, GUEST)",160.0
Massage Room 1,"List(GUEST, GUEST)",160.0
Massage Room 1,"List(GUEST, GUEST)",160.0
Tennis Court 2,"List(GUEST, GUEST)",150.0
Tennis Court 2,"List(GUEST, GUEST)",75.0
Tennis Court 1,"List(GUEST, GUEST)",75.0
Tennis Court 1,"List(GUEST, GUEST)",75.0
Squash Court,"List(GUEST, GUEST)",70.0
Massage Room 1,"List(Jemima, Farrell)",39.6


#### Q9: This time, produce the same result as in Q8, but using a subquery.

In [0]:
%sql
select (select facilities1.name from facilities1), (select firstname from members1) as mem_name, (case when memid=0 then (select guestcost from facilities1) else (select membercost from facilities1) end)*slots as day_cost from bookings1 
where starttime > "2012-09-14" and starttime < "2012-09-15" and (case when memid=0 then (select guestcost from facilities1) else (select membercost from facilities1) end)*slots > 30
order by (case when memid=0 then (select guestcost from facilities1) else (select membercost from facilities1) end)*slots desc

com.databricks.backend.common.rpc.DatabricksExceptions$SQLExecutionException: java.lang.IllegalArgumentException: requirement failed: Subquery subquery#1962, [id=#9724] has not finished
	at scala.Predef$.require(Predef.scala:281)
	at org.apache.spark.sql.execution.ScalarSubquery.doGenCode(subquery.scala:104)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCodeInternal(Expression.scala:182)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.$anonfun$genCodeWithFallback$2(EdgeExpressionCodegen.scala:248)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.$anonfun$genCodeWithFallback$1(EdgeExpressionCodegen.scala:248)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCodeWithFallback(EdgeExpressionCodegen.scala:246)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.doGenCode(EdgeExpressionCodegen.scala:1651)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCode(EdgeExpressionCodegen.scala:196)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.$anonfun$genCodeWithFallback$1(EdgeExpressionCodegen.scala:246)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCodeWithFallback(EdgeExpressionCodegen.scala:246)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.doGenCode(EdgeExpressionCodegen.scala:1777)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCode(EdgeExpressionCodegen.scala:197)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.$anonfun$genCodeWithFallback$1(EdgeExpressionCodegen.scala:246)
	at scala.Option.getOrElse(Option.scala:189)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCodeWithFallback(EdgeExpressionCodegen.scala:246)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.streamlineIRCodeGen(EdgeExpressionCodegen.scala:295)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.streamlineCodeGen(EdgeExpressionCodegen.scala:287)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.doGenCode(EdgeExpressionCodegen.scala:501)
	at com.databricks.sql.expressions.codegen.EdgeExpressionCodegen$.genCode(EdgeExpressionCodegen.scala:76)
	at org.apache.spark.sql.catalyst.expressions.Expression.genCode(Expression.scala:154)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.$anonfun$createOrderKeys$1(GenerateOrdering.scala:76)
	at scala.collection.TraversableLike.$anonfun$map$1(TraversableLike.scala:238)
	at scala.collection.Iterator.foreach(Iterator.scala:941)
	at scala.collection.Iterator.foreach$(Iterator.scala:941)
	at scala.collection.AbstractIterator.foreach(Iterator.scala:1429)
	at scala.collection.IterableLike.foreach(IterableLike.scala:74)
	at scala.collection.IterableLike.foreach$(IterableLike.scala:73)
	at scala.collection.AbstractIterable.foreach(Iterable.scala:56)
	at scala.collection.TraversableLike.map(TraversableLike.scala:238)
	at scala.collection.TraversableLike.map$(TraversableLike.scala:231)
	at scala.collection.AbstractTraversable.map(Traversable.scala:108)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.createOrderKeys(GenerateOrdering.scala:76)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.genComparisons(GenerateOrdering.scala:85)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.create(GenerateOrdering.scala:148)
	at org.apache.spark.sql.catalyst.expressions.codegen.GenerateOrdering$.create(GenerateOrdering.scala:36)
	at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator.generate(CodeGenerator.scala:1563)
	at org.apache.spark.sql.catalyst.expressions.codegen.LazilyGeneratedOrdering. (GenerateOrdering.scala:186)
	at org.apache.spark.sql.catalyst.expressions.codegen.LazilyGeneratedOrdering. (GenerateOrdering.scala:183)
	at org.apache.

#### Q10: Produce a list of facilities with a total revenue less than 1000.
- The output should have facility name and total revenue, sorted by revenue. 
- Remember that there's a different cost for guests and members!

In [0]:
%sql
select facilities1.name, (case when sum((case when bookings1.memid=0 then guestcost else membercost end)*bookings1.slots) < 1000 then sum((case when bookings1.memid=0 then guestcost else membercost end)*bookings1.slots) end) as revenue from bookings1 
join facilities1 on bookings1.facid=facilities1.facid
group by facilities1.name
order by sum((case when bookings1.memid=0 then guestcost else membercost end)*bookings1.slots)

name,revenue
Table Tennis,180.0
Snooker Table,240.0
Pool Table,270.0
Badminton Court,null
Squash Court,null
Tennis Court 1,null
Tennis Court 2,null
Massage Room 2,null
Massage Room 1,null
